In [ ]:
from pyspark.sql.functions import col,from_unixtime,count,when,current_timestamp,from_utc_timestamp

In [ ]:
df = spark.read.table("silver_db.enriched_silverplane_weather_enriched")

In [ ]:
df.show(5)

In [ ]:
df_validated = df.groupBy(from_unixtime(col('time_position')).cast('date').alias('date'),col('country'),col('city'))\
    .agg(count(col('*')).alias('count'),
         count(
             when(col('description').isin('very heavy rain','heavy intensity rain','moderate rain'),1)
               ).alias('flights_in_storm')
        )\
    .withColumn('timestamp',from_utc_timestamp(current_timestamp(), 'Asia/Kolkata'))

In [ ]:
print(df_validated.count())

In [ ]:
try:
    df_validated.write\
        .format('parquet')\
        .mode('overwrite')\
        .option('path','s3://apiproject/gold/weather_Impact_flights/')\
        .saveAsTable('gold_db.weather_Impact_flights')
    print("Table Created gold_db.weather_Impact_flights ")
except Exception as e:
    print(f"Table Creation Failed Error: {e}")